In [1]:
# raw data source: datelazi.ro
 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

raw_data = pd.read_json('https://di5ds1eotmbx1.cloudfront.net/latestData.json')

In [2]:
raw_data

,lasUpdatedOn,lasUpdatedOnString,charts,currentDayStats,historicalData
dailyStats,1602756000,15 Octombrie la 13:00,"{'contains': ['numberInfected', 'numberDecease...",NaN,NaN
quickStats,1602756000,15 Octombrie la 13:00,"{'contains': ['numberInfected', 'numberDecease...",NaN,NaN
counties,1602756000,15 Octombrie la 13:00,"{'contains': ['countyInfectionsNumbers'], 'las...",NaN,NaN
ageHistogram,1602756000,15 Octombrie la 13:00,"{'contains': ['distributionByAge'], 'lastUpdat...",NaN,NaN
averageAge,1602756000,15 Octombrie la 13:00,"{'contains': ['distributionByAge'], 'lastUpdat...",46,NaN
...,...,...,...,...,...
2020-03-21,1602756000,15 Octombrie la 13:00,NaN,NaN,"{'parsedOn': 1584789300, 'parsedOnString': '20..."
2020-03-20,1602756000,15 Octombrie la 13:00,NaN,NaN,"{'parsedOn': 1584702900, 'parsedOnString': '20..."
2020-03-19,1602756000,15 Octombrie la 13:00,NaN,NaN,"{'parsedOn': 1584616500, 'parsedOnString': '20..."
2020-03-18,1602756000,15 Octombrie la 13:00,NaN,NaN,"{'parsedOn': 1584530100, 'parsedOnString': '20..."


In [3]:
# create historical data frame
h_data = raw_data.drop(raw_data[['lasUpdatedOn', 'lasUpdatedOnString', 'charts', 'currentDayStats']], axis=1)
h_data.dropna(inplace=True)

In [4]:
# get most recent data from raw_data
mrd_data = pd.DataFrame.from_dict(raw_data.currentDayStats.countyInfectionsNumbers, orient='Index', columns=['Infections'])
# move county to nonindex column
mrd_data.reset_index(inplace=True)
# add Day column
mrd_data['Day'] = raw_data.currentDayStats.parsedOnString
# rename index -> County column
mrd_data.rename(columns={'index':'County'}, inplace=True)
# convert Day column into date datatype
mrd_data['Day'] = mrd_data['Day'].astype('datetime64')


In [5]:
# create dataframes from per day historical data dictionaries and concatenate them into one df
ch_data = pd.DataFrame()
for t in range(h_data.historicalData.shape[0]):
    ch_data = pd.concat([ch_data, pd.DataFrame(h_data.historicalData[t])], sort=True)

In [6]:
# keep only rows referring to county
ch_data.drop(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '>80', 'în procesare'], inplace=True)

In [7]:
# drop all columns except parsed date and county numbers
ch_data.drop(ch_data.columns.difference(['parsedOnString', 'countyInfectionsNumbers']), axis=1, inplace=True)

In [8]:
# covert parsed date colmn into date dtype
ch_data['parsedOnString'] = ch_data['parsedOnString'].astype('datetime64')

In [9]:
ch_data = ch_data.reset_index()

In [10]:
# rename columns
ch_data.rename(columns={"index": 'County', 'parsedOnString': 'Day', 'countyInfectionsNumbers': 'Infections'}, inplace=True)

In [11]:
# append today's data to ch_data
ch_data = ch_data.append(mrd_data)

In [12]:
# pivot concatenated historical data: counties to columns
pch_data = pd.pivot_table(ch_data, index='Day', columns='County', values='Infections')

In [13]:
# fill NaN values and rename '-' column
pch_data=pch_data.fillna(0)
pch_data.rename(columns={'-': ''}, inplace=True)

In [14]:
# plot county infections evolution on single fig
fig = px.line(pch_data, render_mode='svg', width=1300, height=500)#, range_y=(200,7500))
fig.show()

In [15]:
# calculate infections per day and add to new dataframe
daily_inf_data = pch_data.diff()

In [16]:
# replace NaN values and reset index
daily_inf_data = daily_inf_data.fillna(0).reset_index()

In [17]:
# add totals column (insert at position 2, after NA collumn)
daily_inf_data.insert(2, column='Total', value=daily_inf_data.sum(axis=1, numeric_only=True)) 

In [18]:
# plot daily cases per county and total
fig2 = make_subplots(len(daily_inf_data.columns[2:]), 1, shared_xaxes=True, vertical_spacing=0.004, subplot_titles=daily_inf_data.columns[2:])

for x in range(2, len(daily_inf_data.columns)):
    fig2.add_trace(go.Bar(name=daily_inf_data.columns[x], x=daily_inf_data['Day'], y=daily_inf_data[daily_inf_data.columns[x]]), x-1, 1)

fig2.update_layout(height=4600, width=1300, showlegend=False)
fig2.show()